# NN 2.0

## Defining data functions

Import needed libraries

In [ ]:
# imports
import pandas as pd
import tensorflow as tf
import tensorflow.keras as ks
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

Get data from file and remove columns with text

In [ ]:
def get_swissvotes_data()->pd.DataFrame:
    import re
    
    dataset = pd.read_csv("../data/formatted/swissvotes_dataset_after_1900_utf8.csv", sep=';')
    
    regex = re.compile("pdev_.*")
    to_excl = list(filter(regex.match, dataset.columns))
    
    dataset.drop(columns=to_excl, inplace=True)
    dataset.drop(columns=["legisjahr"], inplace=True)
    dataset.drop(columns=["titel_kurz_d", "titel_kurz_f", "titel_off_d", "titel_off_f", "stichwort"], inplace=True)
    dataset.drop(columns=["swissvoteslink", "anzahl", "anneepolitique", "bkchrono_de", "bkchrono_fr"], inplace=True)
    dataset.drop(columns=["curiavista_de", "curiavista_fr", "urheber", "bkresults_de", "bkresults_fr"], inplace=True)
    dataset.drop(columns=["bfsmap_de", "bfsmap_fr", "nach_cockpit_d", "nach_cockpit_f", "nach_cockpit_e"], inplace=True)
    dataset = dataset[dataset["anr"] < 646] # we don't care about future votes
    
    return dataset
print(f"Defined {get_swissvotes_data}")

In [ ]:
def get_rechtsform_onehot(data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    tensor = tf.one_hot(data["rechtsform"], 5).numpy();
    result = pd.DataFrame(tensor, columns=["ref_obl", "ref_fak", "initiative", "gegen_entw", "stichfr"], index=data.index)
    
    return result.astype(int)
print(f"Defined {get_rechtsform_onehot}")

In [ ]:
def get_politikbereich_multihot(data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    polber = data[["d1e1", "d2e1", "d3e1"]]
    polber = polber.replace('.', 0)
    polber = polber.astype(int)
    
    # the names of the columns (they're a bit long)
    cols = ["Staatsordnung", "Aussenpolitik", "Sicherheitspolitik", "Wirtschaft"]
    cols += ["Landwirtschaft", "Öffentliche Finanzen", "Energie", "Verkehr und Infrastruktur"]
    cols += ["Umwelt und Lebensraum", "Sozialpolitik", "Bildung und Forschung", "Kultur, Religion, Medien"]
    
    result = pd.DataFrame(columns=cols, index = data.index)
    for i in range(len(result)):
        row = np.zeros(12)
        for p in polber.iloc[i]:
            if p != 0:
                row[p-1] = 1
        result.iloc[i] = row
    return result.astype(int)

print(f"Defined {get_politikbereich_multihot}")

In [ ]:
def get_department_onehot(data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    dep_single = data["dep"].replace('.', 2) # voting at age 18 is the only vote with a '.' and it's dep of inner
    dep_single = dep_single.astype(int)
    tensor = tf.one_hot(dep_single, 8).numpy()
    result = pd.DataFrame(tensor, columns=["EDA", "EDI", "EJPD", "VBS", "EFD", "WBF", "UVEK", "BK"], index=data.index)
    
    return result.astype(int)

print(f"Defined {get_department_onehot}")

In [ ]:
def get_bundesrat_onehot(data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    tensor = tf.one_hot(data["br_pos"].replace('.', 3).astype(int), 3).numpy()
    result = pd.DataFrame(tensor, columns=["Dafür_bund", "Dagegen_bund", "Keine_bund"], index=data.index)
    return result.astype(int)

print(f"Defined {get_bundesrat_onehot}")

In [ ]:
def get_legislatur(low:int=1, high:int=10, data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    leg = data["legislatur"]
    def my_map(x:int, x_min:int=leg.min(0), x_max:int=leg.max(0), y_min:int=low, y_max:int=high)->float:
        return (x-x_min)/(x_max-x_min)*(y_max-y_min)+y_min
    
    normalized = data[["legislatur"]].applymap(my_map)
    return normalized

print(f"Defined {get_legislatur}")

In [ ]:
def get_nationalrat_onehot(data:pd.DataFrame = get_swissvotes_data()["nr_pos"])->pd.DataFrame:
    tensor = tf.one_hot(data.astype(int), 3).numpy()
    result = pd.DataFrame(tensor, columns=["Dafür_nat", "Dagegen_nat", "Keine_nat"], index=data.index)
    return result.astype(int)

print(f"Defined {get_nationalrat_onehot}")

In [ ]:
def get_ständerat_onehot(data:pd.DataFrame = get_swissvotes_data()["sr_pos"])->pd.DataFrame:
    tensor = tf.one_hot(data.astype(int), 3).numpy()
    result = pd.DataFrame(tensor, columns=["Dafür_std", "Dagegen_std", "Keine_std"], index=data.index)
    return result.astype(int)

print(f"Defined {get_ständerat_onehot}")

In [ ]:
def get_parties(data:pd.DataFrame = get_swissvotes_data())->list:
    import re
    
    regex_incl = re.compile("p_.*")
    regex_excl = re.compile("p_others_.*")
    
    parties_pre = list(filter(regex_incl.match, data.columns))
    parties = [p for p in parties_pre if not regex_excl.match(p)]
    return parties

print(f"Defined {get_parties}")

In [ ]:
def normalize_party_reco(data:pd.DataFrame = get_swissvotes_data(), names:list = get_parties())->pd.DataFrame:
    # deal with unwanted values first
    normalized = data[names].replace(".", 0)
    normalized.replace(np.nan, 0, inplace=True)
    normalized = normalized.astype(int)
    normalized.replace([3,4,5,66,9999], 0, inplace=True)
    
    result = pd.DataFrame(index=normalized.index)
    
    for p in names: # go through parties and create one hot encoding
        tensor = tf.one_hot(normalized[p], 3).numpy()
        temp = pd.DataFrame(tensor, columns=[p+"_neutral", p+"_ja", p+"_nein"], index=result.index)
        result = result.join(temp)

    return result.astype(int)
print(f"Defined {normalize_party_reco}")

In [ ]:
def get_vote_result(data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    result = data["annahme"].replace('.', 0)
    return result.astype(int)

print(f"Defined {get_vote_result}")

In [ ]:
# creates a dataframe 
def get_canton_results(data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    import re
    regex = re.compile(".*_annahme")
    canton_names = list(filter(regex.match, data.columns))
    return data[canton_names].replace('.', 0).astype(int)

print(f"Defined {get_canton_results}")

In [ ]:
def get_vote_result_proz(data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    result = data["volkja_proz"].replace('.', 0)
    result = result.apply(lambda x: x/100)
    return result.astype(float)

print(f"Defined {get_vote_result_proz}")

In [ ]:
# creates a dataframe 
def get_canton_results_proz(data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    import re
    regex = re.compile(".*_japroz")
    canton_names = list(filter(regex.match, data.columns))
    result = data[canton_names].replace('.', 0)
    result.dropna(inplace=True)
    result = result.apply(lambda x: x/100)
    return result.astype(float)

print(f"Defined {get_canton_results_proz}")

## Training the net

In [ ]:
# get data
def get_data():
    swissvotes = get_swissvotes_data()
    # the inputs used by the neural net are:
        # Rechtsform (one hot),
        # Politikbereich (multi hot),
        # Department (one hot),
        # Position of the Bundesrat (one hot),
        # legislatur (normalized from 1-10),
        # Position of Nationalrat (one hot),
        # Position of Ständerat (one hot),
        # Party recommendations (one hot)
    in_rchtfrm = get_rechtsform_onehot()
    in_poltber = get_politikbereich_multihot()
    in_deprtmt = get_department_onehot()
    in_burapos = get_bundesrat_onehot()
    in_legsltr = get_legislatur()
    in_narapos = get_nationalrat_onehot()
    in_strapos = get_ständerat_onehot()
    in_parties = normalize_party_reco()
    
    inputs = pd.concat([in_rchtfrm, in_poltber, in_deprtmt, in_burapos, in_narapos, in_strapos, in_parties, in_legsltr], axis=1)
    
    # the outputs are:
        # result of the votes (binary),
        # result on a canton level (binary)
    out_result = get_vote_result()
    out_canton = get_canton_results()
    
    outputs = pd.concat([out_result, out_canton], axis=1)
    
    return swissvotes, inputs, outputs

print(f"Defined {get_data}")

In [ ]:
def create_model(input_size:int = len(get_data()[1].columns), hidden:list=[100, 50, 20],
                 output_size:int = len(get_data()[2].columns), activation:str="relu",
                 activation_output:str="sigmoid", 
                 optimizer=ks.optimizers.SGD(learning_rate=0.1), 
                 loss=ks.losses.BinaryCrossentropy())->ks.models.Sequential:
    model = ks.models.Sequential()
    
    model.add(ks.layers.Dense(units=input_size, activation=activation, name="Input"))
    
    for i in range(len(hidden)):
        model.add(ks.layers.Dense(units=hidden[i], activation=activation, name="Hidden_"+str(i)))
        model.add(ks.layers.Dropout(rate=.1, name="Dropout_"+str(i)))
        
    model.add(ks.layers.Dense(units=output_size, activation=activation_output, name="Output"))
    
    model.compile(optimizer=optimizer, loss=loss, metrics=[ks.metrics.BinaryAccuracy(), 
                                                           ks.metrics.FalseNegatives()])
    
    return model

print(f"Defined {create_model}")

In [ ]:
def train_model(model:ks.models.Sequential, inputs:pd.DataFrame=get_data()[1], 
                outputs:pd.DataFrame=get_data()[2], test_size:float=0.2, 
                batch_size:int=50, epochs:int=75, shuffle:bool=True)->tuple:
    from sklearn.model_selection import train_test_split as tts
    in_train, in_test, out_train, out_test = tts(inputs, outputs, test_size=test_size)
    
    history = model.fit(x=in_train, y=out_train, batch_size=batch_size, epochs=epochs, shuffle=shuffle)
    
    return history, in_test, out_test

print(f"Defined {train_model}")

In [ ]:
model = create_model(hidden=[10, 20], 
                     loss=ks.losses.MeanAbsoluteError(), activation_output="sigmoid")

history, in_test, out_test = train_model(model, epochs=125)

print(model.summary())

In [ ]:
model.evaluate(x=in_test, y=out_test)

In [ ]:
def train_classifier(inputs:pd.DataFrame=get_data()[1],
                     outputs:pd.DataFrame=get_data()[2]['annahme'],
                     test_size:float=0.2, cutoff:float=.5,
                     shuffle:bool=False, scale_data:bool=False,
                     visualisation:bool=True, in_train:pd.DataFrame=None,
                     in_test:pd.DataFrame=None, out_train:list=None, out_test:list=None)->tuple:
    if in_train is None or in_test is None or out_train is None or out_test is None:
        from sklearn.model_selection import train_test_split as tts
        cutoff=int(cutoff*len(inputs))
        in_train, in_test, out_train, out_test = tts(inputs[cutoff:], outputs[cutoff:], test_size=test_size, shuffle=shuffle)
    else: assert len(in_train)==len(out_train) and len(in_test)==len(out_test)
    
    if(scale_data):
        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler().fit(in_train)
        in_train = scaler.transform(in_train)
        in_test = scaler.transform(in_test)
    
    out_pred = []
    
    from sklearn.linear_model import SGDClassifier, RidgeClassifier, RidgeClassifierCV
    from xgboost import XGBClassifier
    #from fracridge import FracRidgeRegressor, FracRidgeRegressorCV
    ridge = RidgeClassifier()
    ridge.fit(in_train,out_train)
    out_pred.append([ridge,'ridge', ridge.predict(in_test)])
    
    ridgecv = RidgeClassifierCV()
    ridgecv.fit(in_train,out_train)
    out_pred.append([ridgecv,'ridgecv', ridgecv.predict(in_test)])
    
    #fracridge = FracRidgeRegressor()
    #fracridge.fit(in_train,out_train)
    #out_pred.append([fracridge,'fracridge', fracridge.predict(in_test)])
    
    #fracridgecv = FracRidgeRegressorCV()
    #fracridgecv.fit(in_train,out_train)
    #out_pred.append([fracridge,'fracridge', fracridge.predict(in_test)])
    
    sgd = SGDClassifier(loss='log', penalty='elasticnet')
    sgd.fit(in_train, out_train)
    out_pred.append([sgd,'sgd', sgd.predict(in_test)])
    
    xgb = XGBClassifier()
    xgb.fit(in_train, out_train)
    out_pred.append([xgb,'xgb', xgb.predict(in_test)])
    
    if(visualisation):
        from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
        from xgboost import plot_importance
        errors = []
        errors.append(['r2_score', r2_score])
        errors.append(['mse', mean_squared_error])
        errors.append(['mae', mean_absolute_error])
        
        from sklearn.metrics import plot_confusion_matrix
        for i in out_pred:
            print(i[1], ': ')
            for j in errors:
                print(j[0], j[1](i[2],out_test),' ')
            plot_confusion_matrix(i[0],in_test,out_test)
            print('\n')
    
        fig, ax = plt.subplots(len(out_pred),1, figsize=(20,30))
        axe = ax.ravel()
        for i in range(0,len(out_pred)):
            sns.regplot(ax=axe[i], x=out_pred[i][2], y=out_test, x_bins=100)
            axe[i].set_title(out_pred[i][1])
            axe[i].set_xlabel('recommendations')
            axe[i].set_ylabel('Passed')
        
        plt.rcParams["figure.figsize"] = (15, 20)
        plot_importance(xgb)
        plt.rcParams["figure.figsize"] = (6.4, 4.8)
            
    return out_pred, out_test

In [ ]:
train_classifier(shuffle=True,test_size=0.01,visualisation=False)

In [ ]:
 def get_outcomes_proz():
    out_result_proz = get_vote_result_proz()
    out_canton_proz = get_canton_results_proz()
    
    outputs_proz = pd.concat([out_result_proz, out_canton_proz], axis=1)
    
    return outputs_proz

In [ ]:
def train_regression(inputs:pd.DataFrame=get_data()[1],
                     outputs:pd.DataFrame=get_outcomes_proz()['volkja_proz'],
                     test_size:float=0.2, cutoff:float=.5,
                     shuffle:bool=False, scale_data:bool=False,
                     visualisation:bool=True, in_train:pd.DataFrame=None,
                     in_test:pd.DataFrame=None, out_train:list=None, out_test:list=None)->tuple:
    if in_train is None or in_test is None or out_train is None or out_test is None:
        from sklearn.model_selection import train_test_split as tts
        cutoff=int(cutoff*len(inputs))
        in_train, in_test, out_train, out_test = tts(inputs[cutoff:], outputs[cutoff:], test_size=test_size, shuffle=shuffle)
    else: assert len(in_train)==len(out_train) and len(in_test)==len(out_test)
        
    if(scale_data):
        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler().fit(in_train)
        in_train = scaler.transform(in_train)
        in_test = scaler.transform(in_test)
    
    out_pred = []
    
    from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV
    from xgboost import XGBRegressor
    ridge = Ridge()
    ridge.fit(in_train,out_train)
    out_pred.append([ridge,'ridge', ridge.predict(in_test)])
    
    ridgecv = RidgeCV()
    ridgecv.fit(in_train,out_train)
    out_pred.append([ridgecv,'ridgecv', ridgecv.predict(in_test)])
    
    lasso = Lasso()
    lasso.fit(in_train,out_train)
    out_pred.append([lasso,'lasso', lasso.predict(in_test)])
    
    lassocv = LassoCV()
    lassocv.fit(in_train,out_train)
    out_pred.append([lassocv,'lassocv', lassocv.predict(in_test)])
    
    elasticnet = ElasticNet()
    elasticnet.fit(in_train,out_train)
    out_pred.append([elasticnet,'elasticnet', elasticnet.predict(in_test)])
    
    elasticnetcv = ElasticNetCV()
    elasticnetcv.fit(in_train,out_train)
    out_pred.append([elasticnetcv,'elasticnetcv', elasticnetcv.predict(in_test)])
    
    xgb = XGBRegressor()
    xgb.fit(in_train,out_train)
    out_pred.append([xgb,'xgb', xgb.predict(in_test)])
    
    if(visualisation):
        from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
        from xgboost import plot_importance
        errors = []
        errors.append(['r2_score', r2_score])
        errors.append(['mse', mean_squared_error])
        errors.append(['mae', mean_absolute_error])
        
        for i in out_pred:
            print(i[1], ': ')
            for j in errors:
                print(j[0], j[1](i[2],out_test),' ')
            print('\n')
    
        fig, ax = plt.subplots(len(out_pred),1, figsize=(20,30))
        axe = ax.ravel()
        for i in range(0,len(out_pred)):
            sns.regplot(ax=axe[i], x=out_pred[i][2], y=out_test, x_bins=100)
            axe[i].set_title(out_pred[i][1])
            axe[i].set_xlabel('recommendations')
            axe[i].set_ylabel('Passed')
        
        plt.rcParams["figure.figsize"] = (15, 20)
        plot_importance(xgb)
        plt.rcParams["figure.figsize"] = (6.4, 4.8)
            
    return out_pred, out_test

In [ ]:
train_regression(shuffle=True)

In [ ]:
def plots(inputs:pd.DataFrame=get_data()[1],
          outputs:pd.DataFrame=get_data()[2]):
    fig, ax = plt.subplots(3,1, figsize=(20,30))
    ax[0].hist(inputs['legislatur'], bins=34)
    
    ax[1].hist(outputs['annahme'], bins=34)
    
    ax[2]=sns.lineplot(x=inputs['legislatur'], y=outputs['annahme'])

In [ ]:
plots()

In [ ]:
def iv_classifier(inputs:pd.DataFrame=get_data()[1], middle:pd.DataFrame=get_data()[2].drop(columns=['annahme']),
                  outputs:pd.DataFrame=get_data()[2]['annahme'],test_size:float=0.2, cutoff:float=.5,
                  shuffle:bool=False, scale_data:bool=False,visualisation:bool=True)->tuple:
    from sklearn.model_selection import train_test_split as tts
    cutoff=int(cutoff*len(inputs))
    in_train, in_test, mid_train, mid_test = tts(inputs[cutoff:], middle[cutoff:], test_size=0.01, shuffle=shuffle)
    
    if(scale_data):
        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler().fit(in_train)
        in_train = scaler.transform(in_train)
        in_test = scaler.transform(in_test)
    
    #print(middle.columns)
    mid_pred=[]
    from sklearn.linear_model import SGDClassifier, RidgeClassifier, RidgeClassifierCV
    from xgboost import XGBClassifier
    #from fracridge import FracRidgeRegressor, FracRidgeRegressorCV
    mid_ridge=[]
    for i in middle.columns:
        ridge = RidgeClassifier()
        ridge.fit(in_train,mid_train[i])
        mid_ridge.append(ridge.predict(inputs))
    mid_ridge=np.transpose(mid_ridge)
    mid_pred.append(pd.DataFrame(mid_ridge,columns=middle.columns,index=middle.index))
    
    mid_ridgecv=[]
    for i in middle.columns:
        ridgecv = RidgeClassifierCV()
        ridgecv.fit(in_train,mid_train[i])
        mid_ridgecv.append(ridgecv.predict(inputs))
    mid_ridgecv=np.transpose(mid_ridgecv)
    mid_pred.append(pd.DataFrame(mid_ridgecv,columns=middle.columns,index=middle.index))
    
    #mid_fracrdige=[]
    #for i in middle.columns:
    #    fracridge = FracRidgeRegressor()
    #    fracridge.fit(in_train,mid_train[i])
    #    mid_fracridge.append(fracridge.predict(inputs))
    #mid_fracridge=np.transpose(mid_fracridge)
    #mid_pred.append(pd.DataFrame(mid_fracridge,columns=middle.columns,index=middle.index))
    
    #mid_fracridgecv=[]
    #for i in middle.columns:
    #    fracridgecv = FracRidgeRegressorCV()
    #    fracridgecv.fit(in_train,mid_train[i])
    #    mid_fracridgecv.append(fracridgecv.predict(inputs))
    #mid_fracridgecv=np.transpose(mid_fracridgecv)
    #mid_pred.append(pd.DataFrame(mid_fracridgecv,columns=middle.columns,index=middle.index))
    
    mid_sgd=[]
    for i in middle.columns:
        sgd = SGDClassifier(loss='log', penalty='elasticnet')
        sgd.fit(in_train,mid_train[i])
        mid_sgd.append(sgd.predict(inputs))
    mid_sgd=np.transpose(mid_sgd)
    mid_pred.append(pd.DataFrame(mid_sgd,columns=middle.columns,index=middle.index))
    
    mid_xgb=[]
    for i in middle.columns:
        xgb = XGBClassifier()
        xgb.fit(in_train,mid_train[i])
        mid_xgb.append(xgb.predict(inputs))
    mid_xgb=np.transpose(mid_xgb)
    mid_pred.append(pd.DataFrame(mid_xgb,columns=middle.columns,index=middle.index))
    
    mid_train = [] 
    mid_test = []
    for i in np.arange(0,len(mid_pred)):
        mid_t1, mid_t2 = tts(mid_pred[i][cutoff:], test_size=test_size, shuffle=shuffle)
        mid_train.append(mid_t1)
        mid_test.append(mid_t2)
    out_train, out_test = tts(outputs[cutoff:],test_size=test_size, shuffle=shuffle)
    
    if(scale_data):
        for i in np.arange(0,len(mid_train)):
            scaler = StandardScaler().fit(mid_train[i])
            mid_train[i] = scaler.transform(mid_train[i])
            mid_test[i] = scaler.transform(mid_test[i])
    
    out_pred = []
    
    ridge = RidgeClassifier()
    ridge.fit(mid_train[0],out_train)
    out_pred.append([ridge,'ridge', ridge.predict(mid_test[1])])
    
    ridgecv = RidgeClassifierCV()
    ridgecv.fit(mid_train[1],out_train)
    out_pred.append([ridgecv,'ridgecv', ridgecv.predict(mid_test[1])])
    
    #fracridge = FracRidgeRegressor()
    #fracridge.fit(mid_train[i],out_train)
    #out_pred.append([fracridge,'fracridge', fracridge.predict(mid_test[i])])
    
    #fracridgecv = FracRidgeRegressorCV()
    #fracridgecv.fit(mid_train[i],out_train)
    #out_pred.append([fracridge,'fracridge', fracridge.predict(mid_test[i])])
    
    sgd = SGDClassifier(loss='log', penalty='elasticnet')
    sgd.fit(mid_train[2], out_train)
    out_pred.append([sgd,'sgd', sgd.predict(mid_test[2])])
    
    xgb = XGBClassifier()
    xgb.fit(mid_train[3], out_train)
    out_pred.append([xgb,'xgb', xgb.predict(mid_test[3])])
    
    if(visualisation):
        from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
        from xgboost import plot_importance
        errors = []
        errors.append(['r2_score', r2_score])
        errors.append(['mse', mean_squared_error])
        errors.append(['mae', mean_absolute_error])
        
        for i in out_pred:
            print(i[1], ': ')
            for j in errors:
                print(j[0], j[1](i[2],out_test),' ')
            print('\n')
    
        fig, ax = plt.subplots(len(out_pred),1, figsize=(20,30))
        axe = ax.ravel()
        for i in range(0,len(out_pred)):
            sns.regplot(ax=axe[i], x=out_pred[i][2], y=out_test, x_bins=100)
            axe[i].set_title(out_pred[i][1])
            axe[i].set_xlabel('recommendations')
            axe[i].set_ylabel('Passed')
        
        plt.rcParams["figure.figsize"] = (15, 20)
        plot_importance(xgb)
        plt.rcParams["figure.figsize"] = (6.4, 4.8)
            
    return out_pred, out_test

In [ ]:
iv_classifier(shuffle=False)